In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['PCP']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('phencyclidine', 1127),
 ('planar cell polarity', 741),
 ('pentachlorophenol', 330),
 ('pneumocystis pneumonia', 91),
 ('pneumocystis carinii pneumonia', 56),
 ('peptidyl carrier protein', 40),
 ('primary care physician', 21),
 ('primary care provider', 21),
 ('cell polarity pathway', 15),
 ('perception', 12),
 ('pomegranate concentrate powder', 9),
 ('pulmonary capillary pressure', 9),
 ('pneumocystis jiroveci pneumonia', 8),
 ('precleaved 40 bp primary site containing dna', 8),
 ('procollagen c proteinase', 6),
 ('pentose catabolic pathway', 6),
 ('paracellular permeability', 6),
 ('phenylcyclohexyl piperidine', 6),
 ('papain like cysteine protease', 6),
 ('p carinii pneumonia', 5),
 ('pc pneumonia', 5),
 ('porous coordinated polymer', 4),
 ('poria cocos polysaccharide', 4),
 ('prochelator peptide', 3),
 ('prochlorperazine', 3)]

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [9]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-31 19:34:39] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [10]:
result = [grounding_map, names, pos_labels]

In [11]:
result

[{'cell polarity pathway': 'ungrounded',
  'p carinii pneumonia': 'ungrounded',
  'papain like cysteine protease': 'PF:PF00112)',
  'paracellular permeability': 'ungrounded',
  'pc pneumonia': 'ungrounded',
  'pentachlorophenol': 'CHEBI:CHEBI:17642',
  'pentose catabolic pathway': 'MESH:D010427',
  'peptidyl carrier protein': 'GO:GO:1904091',
  'perception': 'MESH:D010465',
  'phencyclidine': 'CHEBI:CHEBI:8058',
  'phenylcyclohexyl piperidine': 'CHEBI:CHEBI:8058',
  'planar cell polarity': 'MESH:D016764',
  'pneumocystis carinii pneumonia': 'MESH:D011020',
  'pneumocystis jiroveci pneumonia': 'ungrounded',
  'pneumocystis pneumonia': 'MESH:D011020',
  'pomegranate concentrate powder': 'ungrounded',
  'poria cocos polysaccharide': 'ungrounded',
  'porous coordinated polymer': 'ungrounded',
  'precleaved 40 bp primary site containing dna': 'ungrounded',
  'primary care physician': 'MESH:D058007',
  'primary care provider': 'MESH:D058007',
  'prochelator peptide': 'ungrounded',
  'prochlo

In [12]:
grounding_map, names, pos_labels = [{'cell polarity pathway': 'ungrounded',
  'p carinii pneumonia': 'ungrounded',
  'papain like cysteine protease': 'PF:PF00112)',
  'paracellular permeability': 'ungrounded',
  'pc pneumonia': 'ungrounded',
  'pentachlorophenol': 'CHEBI:CHEBI:17642',
  'pentose catabolic pathway': 'MESH:D010427',
  'peptidyl carrier protein': 'GO:GO:1904091',
  'perception': 'MESH:D010465',
  'phencyclidine': 'CHEBI:CHEBI:8058',
  'phenylcyclohexyl piperidine': 'CHEBI:CHEBI:8058',
  'planar cell polarity': 'MESH:D016764',
  'pneumocystis carinii pneumonia': 'MESH:D011020',
  'pneumocystis jiroveci pneumonia': 'ungrounded',
  'pneumocystis pneumonia': 'MESH:D011020',
  'pomegranate concentrate powder': 'ungrounded',
  'poria cocos polysaccharide': 'ungrounded',
  'porous coordinated polymer': 'ungrounded',
  'precleaved 40 bp primary site containing dna': 'ungrounded',
  'primary care physician': 'MESH:D058007',
  'primary care provider': 'MESH:D058007',
  'prochelator peptide': 'ungrounded',
  'prochlorperazine': 'CHEBI:CHEBI:8435',
  'procollagen c proteinase': 'HGNC:8737',
  'pulmonary capillary pressure': 'ungrounded'},
 {'PF:PF00112)': 'Papain family cysteine protease',
  'CHEBI:CHEBI:17642': 'pentachlorophenol',
  'MESH:D010427': 'Pentose Phosphate Pathway',
  'GO:GO:1904091': 'peptidyl carrier protein activity',
  'MESH:D010465': 'Perception',
  'CHEBI:CHEBI:8058': 'phencyclidine',
  'MESH:D016764': 'Cell Polarity',
  'MESH:D011020': 'Pneumonia, Pneumocystis',
  'MESH:D058007': 'Physicians, Primary Care',
  'CHEBI:CHEBI:8435': 'prochlorperazine',
  'HGNC:8737': 'BMP1'},
 ['CHEBI:CHEBI:17642',
  'CHEBI:CHEBI:8058',
  'GO:GO:1904091',
  'HGNC:8737',
  'MESH:D010427',
  'MESH:D011020']]

In [13]:
excluded_longforms = []

In [14]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [17]:
additional_entities = {}

In [18]:
unambiguous_agent_texts = {}

In [19]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [20]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-31 19:54:17] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-31 19:54:56] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9646002508587295 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [21]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:8058': 897,
  'CHEBI:CHEBI:17642': 265,
  'MESH:D011020': 100,
  'ungrounded': 55,
  'HGNC:8737': 6,
  'MESH:D016764': 505,
  'CHEBI:CHEBI:8435': 3,
  'PF:PF00112)': 4,
  'GO:GO:1904091': 33,
  'MESH:D058007': 33,
  'MESH:D010427': 3,
  'MESH:D010465': 1},
 'f1': {'mean': 0.9646, 'std': 0.007461},
 'precision': {'mean': 0.95661, 'std': 0.008289},
 'recall': {'mean': 0.973931, 'std': 0.00744},
 'ungrounded': {'f1': {'mean': 0.362544, 'std': 0.094161},
  'pr': {'mean': 0.272727, 'std': 0.081312},
  'rc': {'mean': 0.56, 'std': 0.135647}},
 'CHEBI:CHEBI:8435': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'PF:PF00112)': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'MESH:D016764': {'f1': {'mean': 0.979501, 'std': 0.006554},
  'pr': {'mean': 0.994059, 'std': 0.011881},
  'rc': {'mean': 0.965504, 'std': 0.009347}},
 'MESH:

In [22]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [23]:
disamb.dump(model_name, results_path)

In [24]:
print(disamb.info())

Disambiguation model for PCP

Produces the disambiguations:
	BMP1*	HGNC:8737
	Cell Polarity	MESH:D016764
	Papain family cysteine protease	PF:PF00112)
	Pentose Phosphate Pathway*	MESH:D010427
	Perception	MESH:D010465
	Physicians, Primary Care	MESH:D058007
	Pneumonia, Pneumocystis*	MESH:D011020
	pentachlorophenol*	CHEBI:CHEBI:17642
	peptidyl carrier protein activity*	GO:GO:1904091
	phencyclidine*	CHEBI:CHEBI:8058
	prochlorperazine	CHEBI:CHEBI:8435

Class level metrics:
--------------------
Grounding                        	Count	F1     
                    phencyclidine*	897	 0.9867
                    Cell Polarity	505	 0.9795
                pentachlorophenol*	265	0.93562
          Pneumonia, Pneumocystis*	100	0.87564
                       Ungrounded	 55	0.36254
peptidyl carrier protein activity*	 33	0.96643
         Physicians, Primary Care	 33	0.91105
                             BMP1*	  6	0.53333
  Papain family cysteine protease	  4	    0.6
                 prochlorperazine	  3	  

In [25]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1